In [ ]:
#!pip install rasterio
#!pip install geopandas
# holis
"""
Created on Wed Mar 29 10:45:35 2023
@author: Alberto
"""

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from shapely.geometry import Polygon
import cv2
from clasificacion_chinchetas import *
import tqdm
import numpy as np
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
import sys
from shapely.geometry import mapping
from shapely import geometry
import cv2
import numpy as np



In [3]:
def matchingTemplate(img_rgb,template,threshold = .7):
    '''
(Function)
(Parameters)
    - threshold: Porcentaje de coincidencia 
    '''
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    w, h = template.shape[::-1]
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    loc = np.where( res >=threshold)
    startpoints=[]
    endpoints=[]
    thresh=[]
    x=-10
    y=-10
    for i,pt in enumerate(zip(*loc[::-1])):
        #if i==0:
        x=pt[0]
        y=pt[1]
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
        cv2.putText(img_rgb,str(i) ,(x,y) , cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        startpoints.append(pt)
        endpoints.append((pt[0] + w, pt[1] + h))
        val=np.array(loc)[:,i]
        thresh.append(res[val[0],val[1]])
        if (pt[0]!=x and pt[0]+1!=x and pt[0]-1!=x and pt[0]+2!=x and pt[0]-2!=x) and (pt[1]!=y and pt[1]+1!=y and pt[1]-1!=y and pt[1]+2!=y and pt[1]-2!=y):
            x=pt[0]
            y=pt[1]
            cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
            cv2.putText(img_rgb,str(i) ,(x,y) , cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
            startpoints.append(pt)
            endpoints.append((pt[0] + w, pt[1] + h))
            val=np.array(loc)[:,i]
            thresh.append(res[val[0],val[1]])
    return img_rgb,startpoints,endpoints,thresh

def Parametro_raster(raster,metros=120):
    """Se obtienen las dimensiones del raster y del cuadro de corte"""
    gdal_interpeter = gdal.Open(raster)
    width = gdal_interpeter.RasterXSize
    height = gdal_interpeter.RasterYSize
    coordenadas_gdal = gdal_interpeter.GetGeoTransform()
    minx = coordenadas_gdal[0]
    miny = coordenadas_gdal[3] + width*coordenadas_gdal[4] + height*coordenadas_gdal[5] 
    maxx = coordenadas_gdal[0] + width*coordenadas_gdal[1] + height*coordenadas_gdal[2]
    maxy = coordenadas_gdal[3]
    src_raster_path = raster
    src=rasterio.open(src_raster_path)
    H,W=src.shape
    dim=int(np.ceil(map_d(minx+metros,minx,maxx,0,W)))
    alto=np.max([1,int(np.floor(H/dim))])
    ancho=np.max([1,int(np.floor(W/dim))])
    return alto,ancho,dim,src.crs,H,W,minx,maxx,miny,maxy

def map_d(x, in_min, in_max, out_min, out_max):
    """Genera una interpolacion para pasar de un rango a otro"""
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def shape_transform(shape):
    """Convierte el shape en dataframe de coordenadas que engloba el polygon del shape para delimitar el raster"""
    c=[]
    angulo_manzana=[]

    # for manzana in range(len(shape)):
    #     proyecciones1=mapping(shape['geometry'][manzana]).get('coordinates')
    #     angulos=[]
    #     d=[]
    #     poly=pd.DataFrame(proyecciones1[0])
    #     for point in range(1,len(poly)):
    #         d.append(((poly[1][point]-poly[1][point-1])**2+(poly[0][point]-poly[0][point-1])**2))
    #         angulos.append(math.atan(((poly[1][point]-poly[1][point-1])/(poly[0][point]-poly[0][point-1])))*180/math.pi)
    #     angulo_manzana.append(angulos[d.index(max(d))])
    shape["angulo_manzana"]=0
    shape["geometry"]=shape["geometry"].envelope
    for manzana in range(len(shape)):
        proyecciones1=mapping(shape['geometry'][manzana]).get('coordinates')
        proyecciones=proyecciones1[0]
        point1=np.min((proyecciones,proyecciones),axis=1)[0]
        min_y,min_x=point1[0],point1[1]
        point2=np.max((proyecciones,proyecciones),axis=1)[0]
        max_y,max_x=point2[0],point2[1]
        c.append(','.join([str(min_y),str(min_x),str(max_y),str(max_x)]))
    shape1=pd.DataFrame()
    shape1['points']=c
    shape1=shape1['points'].str.split(',',expand=True)
    shape1=shape1.astype({0:'float64',1:'float64',2:'float64',3:'float64'})
    shape1["cve_cat"]=shape["cve_cat"]
    shape1["angulo_manzana"]=shape["angulo_manzana"]
    shape=shape1
    return shape

def rotacion_detect(startpoint,endpoint,angle,proyecciones,w,h,dim):
    """Rotata el cuadro detectado en el sistema de coordenadas inicial"""
    point1=np.min((proyecciones,proyecciones),axis=1)[0]
    min_y,min_x=point1[0],point1[1]
    point2=np.max((proyecciones,proyecciones),axis=1)[0]
    max_y,max_x=point2[0],point2[1]
    min_y,min_x,max_y,max_x,proyecciones
    tipos=["casa","terreno"]
    y1,x1=startpoint
    y2,x2=endpoint
    x1,y1=(x1)*2-1,(y1)*2-1
    x2,y2=(x2)*2-1,(y2)*2-1
    angle=angle*math.pi/180
#     x1,y1=x1*(w/dim),y1*(h/dim)
#     x2,y2=x2*(w/dim),y2*(h/dim)
    #x_p, y_p son los puntos de un rectangulo en el orden inverso al manecillas del reloj
    x1p=max_x-((x1*math.cos(angle)-y1*math.sin(angle)+1)/2)*(max_x-min_x)
    y1p=min_y+((x1*math.sin(angle)+y1*math.cos(angle)+1)/2)*(max_y-min_y)
    x2p=max_x-((x2*math.cos(angle)-y1*math.sin(angle)+1)/2)*(max_x-min_x)
    y2p=min_y+((x2*math.sin(angle)+y1*math.cos(angle)+1)/2)*(max_y-min_y)
    x3p=max_x-((x2*math.cos(angle)-y2*math.sin(angle)+1)/2)*(max_x-min_x)
    y3p=min_y+((x2*math.sin(angle)+y2*math.cos(angle)+1)/2)*(max_y-min_y)
    x4p=max_x-((x1*math.cos(angle)-y2*math.sin(angle)+1)/2)*(max_x-min_x)
    y4p=min_y+((x1*math.sin(angle)+y2*math.cos(angle)+1)/2)*(max_y-min_y)
    return Polygon(((y1p,x1p),(y2p,x2p),(y3p,x3p),(y4p,x4p),(y1p,x1p)))

def cv2_imshow(image):
    cv2.namedWindow('name', cv2.WINDOW_NORMAL)
    cv2.imshow('name',image)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [3]:
raster=r"/home/hector/Descargas/temporal/Chinchetas_Ixtapan_centro.tif"
metros = 100
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy=Parametro_raster(raster,metros=metros)
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy

ERROR 1: PROJ: internal_proj_identify: /home/hector/anaconda3/envs/Infis/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


(23,
 43,
 286,
 CRS.from_wkt('LOCAL_CS["WGS 84 / Pseudo-Mercator",UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
 6774,
 12347,
 -11098076.2292,
 -11093754.7532,
 2135322.7351,
 2137693.5448)

In [5]:
# No borrar paths de compañeros

path_hector_nuevo = '/home/hector/Documentos/Infis/Geo/Data/nuevo.png'

template = cv2.imread(path_hector_nuevo)
template.shape
factor = ((6*dim/metros)/21)
temp = cv2.resize(template,(int((factor)*template.shape[1]),int((factor)*template.shape[0])))
# pd.DataFrame(temp[:,:,0]).to_clipboard()

In [8]:
# No borrar paths de compañeros

path_hector = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Ixtapan_de_la_sal/Manzana_Ixtapan_Sal.shp'

shape_o=gpd.read_file(path_hector)
shape_o=shape_o.to_crs('3857')
shape=shape_transform(shape_o.copy())
#shape_aumentado=ampliar_shape(shape_o.copy())

In [9]:
# No borrar path compañeros

path_hector = '/home/hector/Documentos/Infis/Geo/Data/Plantillas'
dict_gris = get_dict_plantilla_gris(path_hector,True,(30,30))


In [11]:
# from google.colab.patches import cv2_imshow

chincheta=[]
clases=[]
threshold=[]
umbrales = []
sizes = []
show_imagen = False
save_detection = False
show_clasification = False
path_save = '/'
template = temp

template_gray=template[:,:,1]
with rasterio.open(raster) as src:
    with tqdm.tqdm(total=alto*ancho*4) as pbar:
        for j in range(ancho*2):#ancho
            for i in range(alto*2):#alto
                generar=0
                label=raster.replace('\\','/').split('/')[-1][:-4]+'_'
                nameimg=label.lower()+str(i)+'_'+str(j)
                cuadro=[]
                for k in range(2):
                    for l in range(2):
                        cuadro.append((minx+(maxx-minx)/ancho*(j/2+k),maxy-(maxy-miny)/alto*(i/2+l)))
                cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
                for punto in cuadro:
                    x=float(punto[0])
                    y=float(punto[1])
                    mini_df=shape[(shape[0]<=x)&(shape[2]>=x)&(shape[1]<=y)&(shape[3]>=y)]
                    if len(mini_df)>0:
                        generar=1
                        #cvees.append(mini_df["cve_cat"].values)### traer todas las cve_catastrales del punto sobre el raster ...Pendiente
#                             print(cvees)
                if generar==1:                    
                    shapes=[{'type':'Polygon','coordinates':[cuadro]}]
                    vector=[]
                    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
                    if np.sum(array)<100:                        
                        pbar.update(1)
                        continue
                    four_images=[array[2],array[1],array[0]]
                    imagen_n = np.stack(four_images, axis=-1)
                    image_o=imagen_n.copy()
                    try:                                               
                        imagen_n,startpoints, endpoints, thresh=matchingTemplate(imagen_n,template_gray,threshold = .7)                        
                    except Exception as e:
                        # print(e)                                              
                        pbar.update(1)
                        continue
                    
                    for s in range(len(startpoints)):
                        startpoint=startpoints[s]                        
                        endpoint=endpoints[s]
                        df_aux=image_o.copy()
                        x1,y1=startpoint
                        x2,y2=endpoint
                        df_aux=df_aux[y1:y2,x1:x2] 
                        
                        # clasificacion
                        clase_aux, n_aux, umbral_aux  = iter_umbral_fn(df_aux, dict_gris,n=100,salto_n=10,
                                                                       umbral=0.94,min_umbral=0.6)
                        
                        # Revisamos la mayor confusion que se da ... areas_verdes ~ establecimiento_google
                        if clase_aux in ['a_verdes','establecimiento_google']:
                            continue
                        
                        # Imprimir detalles de clasificacion                       
                        if show_clasification:
                            print(clase_aux,'  ', n_aux, '  ', umbral_aux)
                            
                        # Apendamos los valores obtenidos
                        clases.append(clase_aux)
                        threshold.append(thresh[s])
                        umbrales.append(umbral_aux)
                        sizes.append(n_aux)
                        
                        # En caso de querer almacenar lo cuadritos, poner ruta
                        if save_detection:
                            cv2.imwrite(path_save +"/"+clase_aux+'_'+str(j)+"_"+str(i)+"_"+str(s)+"deteccion.png",df_aux)

                        startpoint=[x/dim for x in startpoint]
                        endpoint=[x/dim for x in endpoint]
                        proyecciones=cuadro
                        chincheta.append(rotacion_detect(startpoint,endpoint,0,proyecciones,dim,dim,dim))
                        if show_imagen:
                            cv2.imshow('1',df_aux)
                            cv2.waitKey()
                            cv2.destroyAllWindows()

                
                
                pbar.update(1)
                
                

100%|██████████| 3956/3956 [53:32<00:00,  1.23it/s]  


In [12]:
gps=gpd.GeoDataFrame({'Clase':clases,'Threshold':threshold}, geometry=chincheta,crs=3857)
# gps["centroidx"]=gps.centroid.x/10
# gps["centroidy"]=gps.centroid.y/10
# gps["centroidx"]=gps["centroidx"].astype("int")
# gps["centroidy"]=gps["centroidy"].astype("int")
# gps=gps.drop_duplicates(["centroidx","centroidy"])
gps.to_file(r"/home/hector/Descargas/temporal/Ixtapan_centro_chinchetas.shp")

: 

In [ ]:
len(chincheta), len(clases), len(threshold)

In [ ]:
import glob
filenames=glob.glob(r"C:\Users\ASUS\Inteligencia_Artificial\pines/*.png")

In [ ]:
import tqdm
file='C:\\Users\\ASUS\\Inteligencia_Artificial\\chinchetas\\0_28_1deteccion.png'
valor=[]
color=[]
for file in tqdm.tqdm(filenames):
    image2=cv2.imread(file)
    Z = image2.reshape((-1,3))
    # convert to np.float32
    Z = np.float32(Z)
    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 1
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    # Now convert back into uint8, and make original image
    center = np.uint8(center[0])
    color.append("".join([str(int(np.ceil(x/255*10))).zfill(3) for x in center]))
    _,image=cv2.threshold(image2[:,:,1],200,255,cv2.THRESH_BINARY)
    # image=image[5:41,6:29]
    image=image/255
    images=pd.DataFrame(image)
    x1=0
    x2=100
    for col in images.columns:
        if images[col].sum()>=len(images)-5:
            if int(col)>=x1 and int(col)<=int(len(images.columns)/2):
                x1=int(col)+1
            if int(col)>=x1 and int(col)<=x2 and int(col)>=int(len(images.columns)/2):
                x2=int(col)
    y1,y2=(0,len(images))
    image1=image[y1:y2,x1:x2]
    images=pd.DataFrame(image1)
    y1=0
    y2=100
    for i in range(len(images)):
        if images.iloc[i].sum()>=len(images.columns)-5:
            if int(i)>=y1 and int(i)<=int(len(images)/2):
    #             print(y1)
                y1=int(i)+1
            if int(i)>=y1 and int(i)<=y2 and int(i)>=int(len(images)/2):
                y2=int(i)
    image1=image1[y1:y2,0:len(images.columns)]
#     images=pd.DataFrame(image1)
#     x1=0
#     x2=100
#     for col in images.columns:
#         if images[col].sum()==len(images):
#             if int(col)>=x1 and int(col)<=int(len(images.columns)/2):
#                 x1=int(col)+1
#             if int(col)>=x1 and int(col)<=x2 and int(col)>=int(len(images.columns)/2):
#                 x2=int(col)
#     y1,y2=(0,len(images))
#     image1=image1[y1:y2,x1:x2]
#     cv2_imshow("a",image1)
    valor.append(np.sum(image1))
res=pd.DataFrame({"file":filenames,"valor":valor,"color":color})
res

In [ ]:
res["valor1"]=[int(x/10) for x in res["valor"]]
res["valor2"]=[int(str(x).replace(".","")) for x in res["color"]]
Z = np.vstack((res["valor"],res["valor2"]))
# convert to np.float32
Z = np.float32(Z)
# define criteria and apply kmeans()
# criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
# ret,label,center=cv.kmeans(Z,2,None,criteria,10,cv.KMEANS_RANDOM_CENTERS)
# res.groupby("valor2").count()
import matplotlib.pyplot as plt
plt.scatter(res["valor1"],res["valor2"],cmap='viridis',marker="o")
# A = Z[label.ravel()==0]
# B = Z[label.ravel()==1]
# # Plot the data
# plt.scatter(A[:,0],A[:,1])
# plt.scatter(B[:,0],B[:,1],c = 'r')
# plt.scatter(center[:,0],center[:,1],s = 80,c = 'y', marker = 's')
# plt.xlabel('Height'),plt.ylabel('Weight')
# plt.show()

In [ ]:
res["valor2"].value_counts()

In [ ]:
aux=res[res["valor2"]==9009008].sort_values(["valor1","valor2"]).reset_index(drop=True).head(100)
aux

In [ ]:
for i in range(len(aux)):
    im=cv2.imread(aux["file"][i])
    cv2_imshow(str(aux.loc[i,"valor1"])+"_"+str(aux.loc[i,"valor2"]),im)